# Asset Hierarchy Migration with Type

**Prerequisite**:

- Access to a CDF Project.
- Know how to install and setup Python.
- Launch a Python notebook.
- Done the [first part](asset_hierarchy_migration.html) of this tutorial.


In this tutorial we will also show how to migrate an asset hierarchy to a data model representing the same hierarchy in CDF.
However, this will not be a 1-to-1 migration, we will also add classes/types to the assets, such that we get pumps and 
lift-stations in our resulting model.


## Extract Data from Asset Hierarchy

We will use the same asset hierarchy as in the last tutorial

<img src="../../artifacts/figs/asset_hierarchy_lift_pump_stations.png" width="400">

We will go quickly through the steps of setting up a triple store, instantiate the Cognite Client. If you have questions go to the [part 1](asset_hierarchy_migration.html) of this tutorial for a more detailed explaination.

In [18]:
from cognite.neat import get_cognite_client
from cognite.neat.graph import extractors, NeatGraphStore

In [19]:
client = get_cognite_client()
store = NeatGraphStore.from_memory_store()

Found .env file in repository root. Loaded variables from .env file.


Like the first part of this tutorial, we will use the `AssetsExtractor` with the factory method `.from_hierarchy`. However, this time we notice that this
factory method takes in an extra parameter `to_type` which is a function that takes an `Asset` and turns it into type. First, we will write this function
based on our knowlegde of the asset hierarchy.

In [20]:
from cognite.client.data_classes import Asset


def lift_pump_station_to_type(asset: Asset) -> str:
    if asset.external_id.startswith("lift_station"):
        return "LiftStation"
    elif asset.external_id == "lift_pump_stations:root":
        return "Root"
    elif asset.name.lower().startswith("pump"):
        return "Pump"
    else:
        return "Unknown"

Notice in the function above that we utilize our knowlege of the asset hierarchy to infer the type of the asset. First, we know that all `LiftStation`'s external IDs are prefixed with `lift_pump_stations`. For pumps, this does not work as they have an UUID external ID, however, the `name` field always start with `Pump` thus we can utilzie this. Finally, we set the root if the external ID matches the root, and if there is somethign we have missed we set it to `Unknown`. We expect there will be no `unknown`.

In [21]:
asset_extractor = extractors.AssetsExtractor.from_hierarchy(client, root_asset_external_id="lift_pump_stations:root", to_type=lift_pump_station_to_type)

In [22]:
store.write(asset_extractor)

Output()

In [23]:
store

After running the extractor, we see that the type is set to `Pump` for 162 of the assets, `LiftStation` for 82 of them, and we found one root. As expected there were no unknown assets.

## Infering, Exporting and Populating the Data Model

The rest of the tutorial is now the same as part 1.

1. We infer the data model using the `InferenceImporter`.
2. Export the inferred data model to CDF using the `DMSExporter`.
3. Populate it using the `DMSLoader`.

In [24]:
from cognite.neat.rules import importers, exporters
from cognite.neat.graph import loaders

In [25]:
importer = importers.InferenceImporter.from_graph_store(store, prefix="sp_lift_stations")

In [26]:
rules, issues = importer.to_rules()

In [27]:
dms_rules = rules.as_dms_rules()

C:\Users\AndersAlbert\AppData\Local\pypoetry\Cache\virtualenvs\cognite-neat-WszCo0Uu-py3.12\Lib\site-packages\pydantic\main.py:193: MoreThanOneNonAlphanumericCharacterWarning: ('property', 'Shape__Length')
  self.__pydantic_validator__.validate_python(data, self_instance=self)


In [28]:
exporter = exporters.DMSExporter()

In [29]:
result = exporter.export_to_cdf(rules, client)

In [30]:
result

,name,created
0,spaces,1
1,containers,3
2,views,3
3,data_models,1


In [31]:
store.add_rules(rules)

In [32]:
from cognite.client import data_modeling as dm

In [33]:
created = client.data_modeling.spaces.apply(dm.SpaceApply("sp_data_pump_station"))
created

,value
space,sp_data_pump_station
is_global,False
last_updated_time,2024-07-23 07:23:47.330000
created_time,2024-07-23 07:23:47.330000


In [15]:
from cognite.neat.utils.cdf.loaders import SpaceLoader

In [16]:
SpaceLoader(client).clean("sp_lift_stations")

Deleted 3 views
Deleted 3 containers
Deleted 1 data models
Deleted space ['sp_lift_stations']


In [17]:
SpaceLoader(client).clean("sp_data_pump_station")

Deleted space []


In [34]:
loader = loaders.DMSLoader.from_rules(dms_rules, store, instance_space="sp_data_pump_station")

In [35]:
result = loader.load_into_cdf(client)

In [36]:
result

,name,created
0,Nodes,245.0
1,Edges,NaN


## Results

We can now go into CDF and inspect the results. Looking at the data model we created, we can see the schema for
the inferred `Root`, `LiftStation` and `Pump`

<img src="../../artifacts/figs/asset_hierarchy_lift_pump_stations_dms_typed.png" width="400">

Furthermore, we can inspect the populated nodes in this Pump types as well as Lift Station and the Root.

<img src="../../artifacts/figs/asset_hierarchy_lift_pump_stations_populated_with_typed.png" width="1000">